# Partie 2 

## Section 1

In [4]:
import ee

# 1. Initialiser l'API Earth Engine
ee.Authenticate()
ee.Initialize()

# 2. Charger la collection ECMWF/ERA5/DAILY
era5 = ee.ImageCollection("ECMWF/ERA5/DAILY")


Enter verification code:  4/1ASc3gC2dEGeX4F531VfvoMO9gjl8fUEYS2QVzs2nvh4mnGxE1z14_Ref2Fc



Successfully saved authorization token.


In [5]:
# 3. Sélectionner la bande 'total_precipitation'
precip = era5.select("total_precipitation")
# 4. Définir la géométrie correspondant au Ghana
# Utilisation de la collection FAO/GAUL/2015/level0
ghana = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq("ADM0_NAME", "Ghana"))

In [8]:
# 5. Restreindre l'étude à la période 2015-2020
precip_2015_2020 = precip.filterDate("2015-01-01", "2020-12-31")

In [9]:
# 6. Convertir les précipitations journalières en précipitations annuelles moyennes
# Étape A : calculer la somme annuelle
def annual_sum(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, "year")
    yearly = precip_2015_2020.filterDate(start, end).sum()
    return yearly.set("year", year)

years = ee.List.sequence(2015, 2020)
annual_precip = ee.ImageCollection(years.map(annual_sum))

# Étape B : calculer la moyenne annuelle sur la période
mean_annual_precip = annual_precip.mean()

# 7. Réduire sur la géométrie du Ghana pour obtenir une valeur moyenne
ghana_precip = mean_annual_precip.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = ghana.geometry(),
    scale = 10000
)

print("Précipitations annuelles moyennes (2015-2020) au Ghana :",
      ghana_precip.getInfo())

Précipitations annuelles moyennes (2015-2020) au Ghana : {'total_precipitation': 1.0448534210227154}


## Section 2

In [10]:
# Calcul des statistiques globales sur la zone Ghana
stats = mean_annual_precip.reduceRegion(
    reducer = ee.Reducer.mean()
              .combine(reducer2 = ee.Reducer.variance(), sharedInputs=True)
              .combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True),
    geometry = ghana.geometry(),
    scale = 10000,
    bestEffort = True
)

# Extraire les valeurs
mean_val = stats.get("total_precipitation_mean").getInfo()
var_val  = stats.get("total_precipitation_variance").getInfo()
std_val  = stats.get("total_precipitation_stdDev").getInfo()

# Coefficient de variation (CV = std / mean)
cv_val = std_val / mean_val

print("Moyenne annuelle des précipitations :", mean_val)
print("Variance :", var_val)
print("Coefficient de variation :", cv_val)


import geemap

Map = geemap.Map(center=[7.9, -1.0], zoom=6)  # centre du Ghana
Map.addLayer(mean_annual_precip, 
             {"min": 0, "max": 3000, "palette": ["yellow","green","blue"]}, 
             "Précipitations annuelles moyennes")
Map.addLayer(ghana, {}, "Frontières Ghana")
Map


Moyenne annuelle des précipitations : 1.0448534210227154
Variance : 0.0870977410676059
Coefficient de variation : 0.28245422587943336


Map(center=[7.9, -1.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

## Section 3

In [14]:
import ee
ee.Initialize()

# Géométrie du Ghana
ghana = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq("ADM0_NAME", "Ghana"))

# Fonction pour créer une grille régulière
def make_grid(region, scale):
    bounds = region.geometry().bounds()
    grid = bounds.coveringGrid(scale)
    return grid

# Exemple : grille de 50 km
grid = make_grid(ghana, 50000)

# Image des précipitations annuelles moyennes (2015-2020)
era5 = ee.ImageCollection("ECMWF/ERA5/DAILY") \
          .select("total_precipitation") \
          .filterDate("2015-01-01", "2020-12-31")

annual_precip = era5.sum().divide(6)  # somme sur 6 ans puis moyenne

# Associer la moyenne par cellule
def add_precip(feature):
    mean_val = annual_precip.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 10000,
        bestEffort = True
    )
    return feature.set(mean_val)

grid_precip = grid.map(add_precip)

task = ee.batch.Export.table.toDrive(
    collection = grid_precip,
    description = "ghana_precip_grid",
    folder = "GEE",
    fileFormat = "CSV"
)
task.start()